In [184]:
import json
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [185]:
# Load player history (training data)
# df_train = pd.read_csv('data/during-season/player_history.csv', parse_dates=['kickoff_time'])
df_train = pd.read_csv('data/during-season/player_history.csv')

In [186]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6324 entries, 0 to 6323
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   element                          6324 non-null   float64
 1   fixture                          6324 non-null   float64
 2   opponent_team                    6324 non-null   object 
 3   total_points                     6324 non-null   float64
 4   was_home                         6324 non-null   bool   
 5   kickoff_time                     6324 non-null   object 
 6   team_h_score                     6324 non-null   float64
 7   team_a_score                     6324 non-null   float64
 8   round                            6324 non-null   float64
 9   minutes                          6324 non-null   float64
 10  goals_scored                     6324 non-null   float64
 11  assists                          6324 non-null   float64
 12  clean_sheets        

In [187]:
df_train.head(5)

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,recoveries,value,transfers_balance,selected,transfers_in,transfers_out,name,team_id,team,opponent_team_id
0,1.0,6.0,Kalmar FF,6.0,False,2022-04-03T15:30:00Z,0.0,1.0,1.0,90.0,...,5.0,60.0,0.0,4920.0,0.0,0.0,Johan Dahlin,1,Malmö FF,10.0
1,1.0,16.0,IF Elfsborg,3.0,True,2022-04-11T17:10:00Z,1.0,1.0,2.0,90.0,...,6.0,60.0,201.0,5787.0,284.0,83.0,Johan Dahlin,1,Malmö FF,5.0
2,1.0,24.0,AIK,7.0,True,2022-04-17T13:00:00Z,3.0,0.0,3.0,90.0,...,11.0,60.0,39.0,5997.0,127.0,88.0,Johan Dahlin,1,Malmö FF,12.0
3,1.0,32.0,IFK Värnamo,7.0,False,2022-04-21T17:00:00Z,0.0,0.0,4.0,90.0,...,5.0,60.0,241.0,6355.0,319.0,78.0,Johan Dahlin,1,Malmö FF,18.0
4,1.0,39.0,IFK Göteborg,6.0,True,2022-04-25T17:10:00Z,1.0,0.0,5.0,90.0,...,6.0,60.0,144.0,6527.0,244.0,100.0,Johan Dahlin,1,Malmö FF,4.0


In [188]:
df_tmp_X = df_train[['round', 'name', 'team', 'opponent_team', 'was_home']]
train_y = df_train[['total_points']]

In [189]:
df_tmp_X.nunique()

round             14
name             478
team              16
opponent_team     16
was_home           2
dtype: int64

In [190]:
df_tmp_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6324 entries, 0 to 6323
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          6324 non-null   float64
 1   name           6324 non-null   object 
 2   team           6324 non-null   object 
 3   opponent_team  6324 non-null   object 
 4   was_home       6324 non-null   bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 203.9+ KB


In [191]:
df_tmp_X.head()

,round,name,team,opponent_team,was_home
0,1.0,Johan Dahlin,Malmö FF,Kalmar FF,False
1,2.0,Johan Dahlin,Malmö FF,IF Elfsborg,True
2,3.0,Johan Dahlin,Malmö FF,AIK,True
3,4.0,Johan Dahlin,Malmö FF,IFK Värnamo,False
4,5.0,Johan Dahlin,Malmö FF,IFK Göteborg,True


## Preprocessing Data

In [192]:
df_tmp_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6324 entries, 0 to 6323
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   round          6324 non-null   float64
 1   name           6324 non-null   object 
 2   team           6324 non-null   object 
 3   opponent_team  6324 non-null   object 
 4   was_home       6324 non-null   bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 203.9+ KB


In [193]:
def preprocess_col_gw(df):
    # Make column a category
    df['round'] = df['round'].astype('category')
    
    # Let the column know the range and order of the category
    ordered_gws = list(range(1,31))
    df['round'] = df['round'].cat.set_categories(ordered_gws, ordered=True)
    return df

preprocess_col_gw(df_tmp_X)

<ipython-input-193-4dc78b292f67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round'] = df['round'].astype('category')
<ipython-input-193-4dc78b292f67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round'] = df['round'].cat.set_categories(ordered_gws, ordered=True)


,round,name,team,opponent_team,was_home
0,1,Johan Dahlin,Malmö FF,Kalmar FF,False
1,2,Johan Dahlin,Malmö FF,IF Elfsborg,True
2,3,Johan Dahlin,Malmö FF,AIK,True
3,4,Johan Dahlin,Malmö FF,IFK Värnamo,False
4,5,Johan Dahlin,Malmö FF,IFK Göteborg,True
...,...,...,...,...,...
6319,14,Óli Ómarsson,IK Sirius,Degerfors IF,True
6320,14,Niklas Hult,IF Elfsborg,Hammarby,False
6321,14,Fredrik Hammar,Hammarby,IF Elfsborg,True
6322,14,Elyas Bouzaiene,Degerfors IF,IK Sirius,False


In [194]:
df_tmp_X.nunique()

round             14
name             478
team              16
opponent_team     16
was_home           2
dtype: int64

In [195]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

#df_tmp_X['round'] = le.fit_transform(df_tmp_X['round'])

In [196]:
# We need to encode the strings for the scikit model to be able to handle them
# Label encoding for ordered data eg. Low, Medium, High
# OneHot encoding for unordered data
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

def preprocess(df):    
    categorical_features = ["name", "team", "opponent_team"]
    one_hot = OneHotEncoder()
    transformer = ColumnTransformer([("one_hot",
                                      one_hot,
                                     categorical_features)],
                                     remainder="passthrough")

    transformed = transformer.fit_transform(df).toarray()
    transformed = pd.DataFrame(transformed)
    return transformed

transformed_X = preprocess(df_tmp_X)
transformed_X

/usr/local/Caskroom/miniconda/base/envs/env/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,1.0
6320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
6321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,1.0
6322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,14.0,0.0


In [197]:
# Check number of unique values in columns, just to compare against transformed df
print(df_tmp_X.apply(lambda col: col.nunique()))

round             14
name             478
team              16
opponent_team     16
was_home           2
dtype: int64


In [198]:
# Ok, appearantly, there is an easier way to do that.. but we will skip this for now
dummies = pd.get_dummies(df_tmp_X[["name", "team", "opponent_team"]])
dummies

,name_Abbe Khalili,name_Abdelrahman Saidi,name_Abdul Malik Abubakari,name_Abdussalam Magashy,name_Adam Bergmark Wiberg,name_Adam Carlén,name_Adam Hellborg,name_Adam Ingi Benediktsson,name_Adam Kaied,name_Adam Petersson,...,opponent_team_Helsingborgs IF,opponent_team_IF Elfsborg,opponent_team_IFK Göteborg,opponent_team_IFK Norrköping,opponent_team_IFK Värnamo,opponent_team_IK Sirius,opponent_team_Kalmar FF,opponent_team_Malmö FF,opponent_team_Mjällby AIF,opponent_team_Varbergs BoIS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6319,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6320,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6321,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [199]:
# Convert strings into pandas category codes since the regressor model cannot handle strings
# for label, content in df_tmp.items():
    # if pd.api.types.is_string_dtype(content):
        # df_tmp[label] = content.astype("category")
#         df_tmp[label] = pd.Categorical(content).codes

In [200]:
# transformed.to_csv('data/during-season/df_transformed.csv', index=False)

In [201]:
# df_tmp = pd.read_csv('data/during-season/df_tmp.csv')

## Build model

In [202]:
train_X = transformed_X

In [203]:
%%time
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, random_state=1234)
model.fit(train_X, train_y.values.ravel())

CPU times: user 7min 35s, sys: 1.06 s, total: 7min 36s
Wall time: 59.3 s


RandomForestRegressor(n_jobs=-1, random_state=1234)

In [204]:
def plot_feature_importance(columns, importances):
    df = (pd.DataFrame({'features': columns, 'feature_importances': importances})
          .sort_values('feature_importances', ascending=False))
    
    fig, ax = plt.subplots()
    ax.barh(df['features'], df['feature_importances'])
    ax.set_ylabel("Features")
    ax.set_xlabel("Feature importance")
    ax.invert_yaxis()

In [205]:
# plot_feature_importance(df_tmp_X.columns, model.feature_importances_)

In [206]:
df_tmp_X.shape

(6324, 5)

In [207]:
df_tmp_X.sample(5)

,round,name,team,opponent_team,was_home
333,4,Samuel Adrian,Malmö FF,IFK Värnamo,False
1301,1,Viktor Lundberg,Helsingborgs IF,Hammarby,False
1932,2,Johan Karlsson,IK Sirius,IFK Värnamo,False
2803,5,Besfort Zeneli,IF Elfsborg,GIF Sundsvall,True
259,5,Romain Gall,Malmö FF,IFK Göteborg,True


In [208]:
df_tmp_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6324 entries, 0 to 6323
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   round          6324 non-null   category
 1   name           6324 non-null   object  
 2   team           6324 non-null   object  
 3   opponent_team  6324 non-null   object  
 4   was_home       6324 non-null   bool    
dtypes: bool(1), category(1), object(3)
memory usage: 162.0+ KB


In [209]:
import numpy as np
import scipy as sp
from scipy.cluster import hierarchy as hc
import seaborn as sns

def cluster_columns(df, figsize=(10,6), font_size=12):
    corr = sp.stats.spearmanr(df).correlation
    
    fix, ax = plt.subplots(figsize=(10, 15))
    ax = sns.heatmap(corr, annot=True, linewidths=0.5, fmt='.3f', cmap='YlGnBu')
    plt.show()
    
# cluster_columns(df_tmp_X)

In [210]:
from scipy.cluster.hierarchy import dendrogram, linkage

def cluster_columns(df):
    print(df.shape)
    linked = linkage(df)
    print(linked.shape)
    
    
# cluster_columns(df_tmp_X)

In [211]:
import numpy as np
from scipy.cluster import hierarchy as hc
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram


def cluster_columns(df, figsize=(20,16), font_size=12):
    corr = np.round(sp.stats.spearmanr(df).correlation, 4)
    corr_condensed = hc.distance.squareform(1-corr)
    z = hc.linkage(corr_condensed, method='average')
    fig = plt.figure(figsize=figsize)
    hc.dendrogram(z, labels=df.columns, orientation='left', leaf_font_size=font_size)
    plt.show()
    
# cluster_columns(df_tmp_X)

## Predict

In [212]:
# Show train data to see correct format
df_tmp_X.head()

,round,name,team,opponent_team,was_home
0,1,Johan Dahlin,Malmö FF,Kalmar FF,False
1,2,Johan Dahlin,Malmö FF,IF Elfsborg,True
2,3,Johan Dahlin,Malmö FF,AIK,True
3,4,Johan Dahlin,Malmö FF,IFK Värnamo,False
4,5,Johan Dahlin,Malmö FF,IFK Göteborg,True


### Get data to predict

In [213]:
df_all_player_fixtures = pd.read_csv('data/during-season/player_fixtures.csv')

In [214]:
# Next gameweek to predict
gw_to_predict = df_all_player_fixtures['event'].min()

In [215]:
df_predict = df_all_player_fixtures[['event', 'name', 'team', 'opponent_team', 'is_home']]
df_predict = df_predict.rename(columns={'event': 'round', 'is_home': 'was_home'})
df_predict = df_predict[df_predict['round'] == gw_to_predict]
df_predict.head(20)

,round,name,team,opponent_team,was_home
0,15,Johan Dahlin,Malmö FF,IK Sirius,True
15,15,Ismael Diawara,Malmö FF,IK Sirius,True
30,15,Niklas Moisander,Malmö FF,IK Sirius,True
45,15,Jonas Knudsen,Malmö FF,IK Sirius,True
60,15,Lasse Nielsen,Malmö FF,IK Sirius,True
75,15,Eric Larsson,Malmö FF,IK Sirius,True
90,15,Felix Beijmo,Malmö FF,IK Sirius,True
105,15,Martin Olsson,Malmö FF,IK Sirius,True
120,15,Matej Chalus,Malmö FF,IK Sirius,True
135,15,Markus Björkqvist,Malmö FF,IK Sirius,True


In [216]:
preprocess_col_gw(df_predict)
df_predict['round']

0       15
15      15
30      15
45      15
60      15
        ..
7496    15
7512    15
7528    15
7544    15
7559    15
Name: round, Length: 481, dtype: category
Categories (30, int64): [1 < 2 < 3 < 4 ... 27 < 28 < 29 < 30]

In [217]:
df_tmp_X.nunique()

round             14
name             478
team              16
opponent_team     16
was_home           2
dtype: int64

In [218]:
df_predict.nunique()

round              1
name             480
team              16
opponent_team     16
was_home           2
dtype: int64

In [219]:
# Check if there are any new players that we are not able to predict yet since they have no game history
num_training_players = df_tmp_X['name'].nunique()
print("Unique player names in training data:", num_training_players)
num_predict_players = df_predict['name'].nunique()
print("Unique player names in predict data:", num_predict_players)
if (num_training_players != num_predict_players):
    print("\n⚠️ Number of players in predicted data but not in training:", num_predict_players - num_training_players)
    
    missing_player_index = np.where(np.isin(df_predict['name'], df_tmp_X['name']) == False)
    missing_players_in_training = df_predict['name'].unique()[missing_player_index]
    print("Missing players in training data:", ", ".join(missing_players_in_training))

    print("Removing those players from predict data...")
    df_predict = df_predict[df_predict['name'].isin(df_tmp_X['name'].unique())]
    print("Unique player names in predict data after cleanup:", df_predict['name'].nunique())

Unique player names in training data: 478
Unique player names in predict data: 480

⚠️ Number of players in predicted data but not in training: 2
Missing players in training data: Marcus Danielson, Anton Nilsson
Removing those players from predict data...
Unique player names in predict data after cleanup: 478


In [220]:
df_predict_transformed = preprocess(df_predict)

/usr/local/Caskroom/miniconda/base/envs/env/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


### Make prediction

In [221]:
predicted = model.predict(df_predict_transformed)

In [222]:
df_predicted = pd.DataFrame(predicted)
df_predicted

,0
0,4.30
1,0.20
2,1.44
3,0.12
4,6.15
...,...
474,0.35
475,0.00
476,0.02
477,0.31


In [223]:
df_predict.reset_index(drop=True, inplace=True)
df_predicted.reset_index(drop=True, inplace=True)
df_result = df_predict.join(df_predicted)
df_result.rename(columns={0: 'predicted_points'}, inplace=True)
df_result.sort_values(by='predicted_points', ascending=False, inplace=True)

# Print the top predicted points for round
df_result.head(20)

,round,name,team,opponent_team,was_home,predicted_points
251,15,Victor Edvardsen,Djurgården,BK Häcken,False,9.05
224,15,Alexander Jeremejeff,BK Häcken,Djurgården,True,8.16
123,15,Gustav Ludwigson,Hammarby,Varbergs BoIS,False,8.15
138,15,Moustafa Zeidan,Malmö FF,IK Sirius,True,7.64
230,15,Hjalmar Ekdal,Djurgården,BK Häcken,False,6.88
245,15,Magnus Eriksson,Djurgården,BK Häcken,False,6.71
126,15,Nahir Besara,Hammarby,Varbergs BoIS,False,6.66
111,15,Simon Sandberg,Hammarby,Varbergs BoIS,False,6.57
6,15,Felix Beijmo,Malmö FF,IK Sirius,True,6.47
4,15,Lasse Nielsen,Malmö FF,IK Sirius,True,6.15


In [224]:
df_result.tail()

,round,name,team,opponent_team,was_home,predicted_points
235,15,Elliot Käck,Djurgården,BK Häcken,False,0.0
236,15,Melker Jonsson,Djurgården,BK Häcken,False,0.0
237,15,Axel Wallenborg,Djurgården,BK Häcken,False,0.0
248,15,Albion Ademi,Djurgården,BK Häcken,False,0.0
478,15,Anton Nilsson,Helsingborgs IF,GIF Sundsvall,True,0.0


### Add "predicted point/cost"

# TODO: This fails because the indexing is based on player name and there are players with the same name. Indexing needs to be based on the player id instead.

In [225]:
# Get latest price for every player

# Load all player season history into a single data frame
with open('data/during-season/bootstrap-static.json', 'r') as json_file:
    data = json.loads(json_file.read())    
df_player_overview = pd.json_normalize(data, record_path='elements')
df_player_overview['name'] = df_player_overview["first_name"] + ' ' + df_player_overview["second_name"]

# df_player_overview
# df_result['price (M)'] = df_result['name'].map(df_player_overview['now_cost']/10)
# df_result

# Calculate points/price
# df_result['predicted_point_per_price'] = df_result['predicted_points']/df_result['price (M)']
# df_result.sort_values(by=['predicted_point_per_price'], ascending=False, inplace=True)